# WHO Air quality database

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import re

## Prepare Data

In [94]:
def Trim_Region(value : str)->str:
    r"""
    summary:
        remove LMIC, HIC, and Sub-Saharan
    """
    return re.sub('\s*(\(HIC\)|\(LMIC\)|\(Sub-Saharan\))', '', value)


def CalculateValues(orgvalue)->int:
    strvalue = str(orgvalue)
    match = re.match(r'\((\d+)\)-converted value', strvalue)
    if match:
        return int(match.group(1))
    return int(strvalue)
def GetIncome(Region : str)->str:
    return re.match('.+\((\w+)\)$', str(Region)).group(1)


data = pd.read_csv('database.csv', sep=';', skiprows=2)
data = data.drop(['status', 'Database version (year)', 'Reference for air quality', 'iso3'], axis = 1)

data['Income'] = [GetIncome(value) for value in data.Region]
data['Region'] = [Trim_Region(value) for value in data.Region]

pm10data= data.filter(['Region', 'Country', 'City/Town', 'Year', 'Annual mean, ug/m3', 'Temporal coverage', 'note on converted PM10', 'Number and type of monitoring stations', 'Income'], axis=1)
pm25data= data.filter(['Region', 'Country', 'City/Town', 'Year', 'Annual mean, ug/m3.1', 'Temporal coverage.1', 'note on converted PM2.5', 'Number and type of monitoring stations', 'Income'], axis=1)
pm25data= pm25data.rename(columns={ 'Temporal coverage.1' : 'Temporal coverage'})
pm10data['Convered'] = pm10data['note on converted PM10'].str.contains("Converted")
pm10data['Value'] = [CalculateValues(value) for value in pm10data['Annual mean, ug/m3']]
pm10data = pm10data.drop(['note on converted PM10', 'Annual mean, ug/m3'], axis = 1)
pm25data['Convered'] = pm25data['note on converted PM2.5'].str.contains("Converted")
pm25data['Value'] = [CalculateValues(value) for value in pm25data['Annual mean, ug/m3.1']]
pm25data = pm25data.drop(['note on converted PM2.5', 'Annual mean, ug/m3.1'], axis = 1)

## Get Country and City Information

In [104]:
def GetCountries()->list:
    return data.Country.unique()

def GetCities()->list:
    return data['City/Town'].unique()

In [105]:
len(GetCities())

4468